In [1]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.python.lib.io import file_io


import keras
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

%matplotlib inline

Using TensorFlow backend.


In [11]:
EPOCHS = 300
BS = 256
DROPOUT_RATE = 0.5
SGD_LEARNING_RATE = 0.01

#data = pd.read_csv('fer2013/icml_face_data.csv', sep=r'\s,\s', header=0, encoding='ascii', engine='python') 
data = pd.read_csv('fer2013/icml_face_data.csv') 
data.head()

data.columns

data_train = data[data[' Usage'] == 'Training']
#print('Number samples in the training dataset: ', data_train.shape[0])

data_dev = data[data[' Usage'] == 'PublicTest']
#print('Number samples in the development dataset: ', data_dev.shape[0]) print(data_dev.head())

data_test = data[data[' Usage'] == 'PrivateTest']
#print('Number samples in the development dataset: ', data_dev.shape[0]) print(data_test.head())

data_train.to_csv('fer2013/train.csv') 
data_dev.to_csv('fer2013/dev.csv') 
data_test.to_csv('fer2013/test.csv')

print(data_train.shape) 
print(data_dev.shape) 
print(data_test.shape)

In [4]:
# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
    # dataset: Data path
def get_data(dataset):
    
    file_stream = file_io.FileIO(dataset, mode='r')
    data = pd.read_csv(file_stream)

    #data = pd.read_csv('fer2013/fer2013.csv')
    data[' pixels'] = data[' pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])

    # Retrieve train input and target
    X, Y = data[' pixels'].tolist(), data['emotion'].values
    
    # Reshape images to 4D (num_samples, width, height, num_channels)
    X_res = np.array(X, dtype='float32').reshape(-1,48,48,1)
    # Normalize images with max (the maximum pixel intensity is 255)
    X_res = X_res/255.0
    #image_resized = resize(image, (image.shape[0] // 4, image.shape[1] // 4), anti_aliasing=True)

    Y_res = np.zeros((Y.size, 7))
    Y_res[np.arange(Y.size),Y] = 1    
    
    return  X_res, Y_res

In [5]:
training_dataset_dir = 'fer2013/train.csv'
dev_dataset_dir = 'fer2013/dev.csv'
# Data preparation
X_train, Y_train  = get_data(training_dataset_dir)
X_dev, Y_dev      = get_data(dev_dataset_dir)

# Generate batches of tensor image data with real-time data augmentation. The data will be looped over (in batches) indefinitely
# rescale:          Rescaling factor (defaults to None). Multiply the data by the value provided (before applying any other transformation)
# rotation_range:   Int. Degree range for random rotations
# shear_range:      Float. Shear Intensity (Shear angle in counter-clockwise direction as radians)
# zoom_range:       Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range]
# fill_mode :       Points outside the boundaries of the input are filled according to the given mode: {"constant", "nearest", "reflect" or "wrap"}
# horizontal_flip:  Boolean. Randomly flip inputs horizontally
train_datagen = ImageDataGenerator(
    rotation_range  = 10,
    shear_range     = 5, # 10 degrees
    zoom_range      = 0.1,
    fill_mode       = 'reflect',
    horizontal_flip = True)

# Takes numpy data & label arrays, and generates batches of augmented/normalized data. Yields batcfillhes indefinitely, in an infinite loop
    # x:            Data. Should have rank 4. In case of grayscale data, the channels axis should have value 1, and in case of RGB data, 
    #               it should have value 3
    # y:            Labels
    # batch_size:   Int (default: 32)
train_generator = train_datagen.flow(X_train, Y_train,  batch_size  = BS)

In [6]:
# Implement below paper CPCPCPFF depth 5, 2.4m params
# http://openaccess.thecvf.com/content_cvpr_2016_workshops/w28/papers/Kim_Fusing_Aligned_and_CVPR_2016_paper.pdf
# Reference: https://arxiv.org/pdf/1612.02903.pdf
model = Sequential()
model.add(Conv2D(64, (5, 5), activation='relu',padding='same', input_shape=(48,48,1),name="conv1"))
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool1"))
model.add(Dropout(0.5))
model.add(Conv2D(96, (5, 5), activation='relu',padding='same',name="conv2"))
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool2"))         
model.add(Dropout(0.5))
model.add(Conv2D(256, (5, 5), activation='relu',padding='same',name="conv3"))
model.add(MaxPooling2D(pool_size=(2, 2),name="maxpool3"))
model.add(Conv2D(256, (5, 5), activation='relu',padding='same',name="conv4"))

model.add(Flatten())
model.add(Dense(2048, activation='relu',name='fc1'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax',name='fcsoftmax'))

#TODO: weight decay of 0.0001...initial learning rate is set to 0.01 and reduced by a factor of 2 at every 25 epoch
sgd = SGD(lr=0.01,momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [7]:
model.fit_generator(
    generator           = train_generator,
    validation_data=(X_dev, Y_dev), 
    steps_per_epoch=len(X_train) // BS,
    shuffle=True,
    epochs=EPOCHS) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/300





112/112 [==============================] - 31s 273ms/step - loss: 1.8229 - acc: 0.2475 - val_loss: 1.8408 - val_acc: 0.2494
Epoch 2/300
112/112 [==============================] - 8s 73ms/step - loss: 1.7963 - acc: 0.2529 - val_loss: 1.8246 - val_acc: 0.2435
Epoch 3/300
112/112 [==============================] - 8s 74ms/step - loss: 1.7681 - acc: 0.2680 - val_loss: 1.7777 - val_acc: 0.2747
Epoch 4/300
112/112 [==============================] - 8s 74ms/step - loss: 1.7386 - acc: 0.2955 - val_loss: 1.7289 - val_acc: 0.3188
Epoch 5/300
112/112 [==============================] - 8s 75ms/step - loss: 1.7106 - acc: 0.3151 - val_loss: 1.7116 - val_acc: 0.3101
Epoch 6/300
112/112 [==============================] - 8s 75ms/step - loss: 1.6803 - acc: 0.3347 - val_loss: 1.6934 - val_acc: 0.3238
Epoch 7/300
112/112 [==============================] - 8s 74ms/step - loss: 1.6452 - acc: 0.

Epoch 46/300
112/112 [==============================] - 8s 75ms/step - loss: 1.0238 - acc: 0.6123 - val_loss: 1.1054 - val_acc: 0.5913
Epoch 47/300
112/112 [==============================] - 8s 74ms/step - loss: 1.0127 - acc: 0.6227 - val_loss: 1.1048 - val_acc: 0.5921
Epoch 48/300
112/112 [==============================] - 8s 74ms/step - loss: 1.0100 - acc: 0.6213 - val_loss: 1.0936 - val_acc: 0.5982
Epoch 49/300
112/112 [==============================] - 8s 76ms/step - loss: 0.9836 - acc: 0.6351 - val_loss: 1.0917 - val_acc: 0.5991
Epoch 50/300
112/112 [==============================] - 8s 75ms/step - loss: 0.9730 - acc: 0.6383 - val_loss: 1.1172 - val_acc: 0.6004
Epoch 51/300
112/112 [==============================] - 8s 74ms/step - loss: 0.9600 - acc: 0.6394 - val_loss: 1.0959 - val_acc: 0.6060
Epoch 52/300
112/112 [==============================] - 9s 76ms/step - loss: 0.9474 - acc: 0.6460 - val_loss: 1.1138 - val_acc: 0.5974
Epoch 53/300
112/112 [==============================] -

112/112 [==============================] - 9s 76ms/step - loss: 0.3651 - acc: 0.8677 - val_loss: 1.4176 - val_acc: 0.6194
Epoch 107/300
112/112 [==============================] - 9s 76ms/step - loss: 0.3592 - acc: 0.8701 - val_loss: 1.4187 - val_acc: 0.6233
Epoch 108/300
112/112 [==============================] - 9s 77ms/step - loss: 0.3518 - acc: 0.8751 - val_loss: 1.4323 - val_acc: 0.6213
Epoch 109/300
112/112 [==============================] - 8s 75ms/step - loss: 0.3505 - acc: 0.8756 - val_loss: 1.3860 - val_acc: 0.6317
Epoch 110/300
112/112 [==============================] - 9s 76ms/step - loss: 0.3529 - acc: 0.8741 - val_loss: 1.4674 - val_acc: 0.6314
Epoch 111/300
112/112 [==============================] - 8s 75ms/step - loss: 0.3334 - acc: 0.8831 - val_loss: 1.4647 - val_acc: 0.6211
Epoch 112/300
112/112 [==============================] - 8s 75ms/step - loss: 0.3371 - acc: 0.8794 - val_loss: 1.4625 - val_acc: 0.6311
Epoch 113/300
112/112 [==============================] - 9s 77

112/112 [==============================] - 9s 76ms/step - loss: 0.1649 - acc: 0.9430 - val_loss: 1.7626 - val_acc: 0.6236
Epoch 167/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1674 - acc: 0.9423 - val_loss: 1.7484 - val_acc: 0.6317
Epoch 168/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1617 - acc: 0.9442 - val_loss: 1.7193 - val_acc: 0.6361
Epoch 169/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1569 - acc: 0.9467 - val_loss: 1.8238 - val_acc: 0.6283
Epoch 170/300
112/112 [==============================] - 9s 76ms/step - loss: 0.1548 - acc: 0.9479 - val_loss: 1.7854 - val_acc: 0.6272
Epoch 171/300
112/112 [==============================] - 8s 76ms/step - loss: 0.1581 - acc: 0.9459 - val_loss: 1.8042 - val_acc: 0.6252
Epoch 172/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1556 - acc: 0.9464 - val_loss: 1.7008 - val_acc: 0.6289
Epoch 173/300
112/112 [==============================] - 8s 75

112/112 [==============================] - 9s 77ms/step - loss: 0.1065 - acc: 0.9632 - val_loss: 1.8614 - val_acc: 0.6333
Epoch 227/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1045 - acc: 0.9648 - val_loss: 1.8656 - val_acc: 0.6325
Epoch 228/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1116 - acc: 0.9632 - val_loss: 1.8462 - val_acc: 0.6308
Epoch 229/300
112/112 [==============================] - 8s 74ms/step - loss: 0.1033 - acc: 0.9635 - val_loss: 1.8930 - val_acc: 0.6350
Epoch 230/300
112/112 [==============================] - 8s 75ms/step - loss: 0.0982 - acc: 0.9670 - val_loss: 1.9286 - val_acc: 0.6311
Epoch 231/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1034 - acc: 0.9661 - val_loss: 1.9309 - val_acc: 0.6294
Epoch 232/300
112/112 [==============================] - 8s 75ms/step - loss: 0.1033 - acc: 0.9666 - val_loss: 1.9326 - val_acc: 0.6328
Epoch 233/300
112/112 [==============================] - 9s 76

112/112 [==============================] - 9s 76ms/step - loss: 0.0746 - acc: 0.9756 - val_loss: 2.0883 - val_acc: 0.6291
Epoch 287/300
112/112 [==============================] - 8s 75ms/step - loss: 0.0875 - acc: 0.9704 - val_loss: 2.0304 - val_acc: 0.6294
Epoch 288/300
112/112 [==============================] - 8s 75ms/step - loss: 0.0823 - acc: 0.9725 - val_loss: 1.9992 - val_acc: 0.6342
Epoch 289/300
112/112 [==============================] - 8s 76ms/step - loss: 0.0807 - acc: 0.9723 - val_loss: 2.0203 - val_acc: 0.6406
Epoch 290/300
112/112 [==============================] - 8s 75ms/step - loss: 0.0760 - acc: 0.9732 - val_loss: 2.0142 - val_acc: 0.6317
Epoch 291/300
112/112 [==============================] - 9s 76ms/step - loss: 0.0728 - acc: 0.9759 - val_loss: 2.0649 - val_acc: 0.6367
Epoch 292/300
112/112 [==============================] - 8s 75ms/step - loss: 0.0761 - acc: 0.9744 - val_loss: 2.0515 - val_acc: 0.6364
Epoch 294/300
112/112 [==============================] - 8s 75

In [8]:
#model.fit(X_train,y_train,batch_size=500,epochs=1000,validation_data=(X_dev, y_dev))

In [9]:
print('\n# Evaluate on dev data')
results_dev = model.evaluate(X_dev, Y_dev , batch_size=128)
print('dev loss, dev acc:', results_dev)


# Evaluate on test data
3589/3589 [==============================] - 0s 61us/step
test loss, test acc: [2.087482804440296, 0.6372248537280029]


In [10]:
test_dataset_dir = 'fer2013/test.csv'
X_test, Y_test      = get_data(test_dataset_dir)

print('\n# Evaluate on test data')
results_test = model.evaluate(X_test, Y_test , batch_size=128)
print('test loss, test acc:', results_test)


# Evaluate on test data
3589/3589 [==============================] - 0s 39us/step
test loss, test acc: [1.9984358973608072, 0.6386179994759569]


In [13]:
lr_str = '-SGD_LR_%.5f' % SGD_LEARNING_RATE
epoch_str = '-EPOCHS_' + str(EPOCHS)
bs_str = '-BS_' + str(BS)
dropout_str = '-DROPOUT_' + str(DROPOUT_RATE)
model.save('models/soa' + lr_str + epoch_str + bs_str + dropout_str + '.h5')